# [CAADRIA 2019] Multi Objective Evolutionary Algorithms - PF processing 

In [1]:
# https://plot.ly/python/axes/
import pandas as pd
import numpy as np

In [2]:
# Files Configurations
files_header = False
base_folder = "final/outputs/algorithms/"

# Get the data (csv file should be on the same folder as this file)
def get_algorithm_data(algorithm, parent_folder=base_folder):
    file = parent_folder + "all_" + algorithm + "_solutions.csv"
    print("Reading file: " + file)
    return pd.read_csv(file, sep=',', header=None)

In [3]:
# Variables and Objective columns
vars_cols = [3, 4, 5, 6, 7, 8]
objs_cols = [9, 10]

# Drop columns (Time total, t1, t2)
drop_cols = [0, 1, 2]

In [4]:
# Objectives
o1 = objs_cols[0]
o2 = objs_cols[1]

In [5]:
# Algorithms 
# algorithms = ["MOEAD", "SPEA2", "NSGAII", "OMOPSO", "SMPSO", "PESA2", "PAES", "EpsMOEA", "GDE3", "CMAES"]
algorithms = ["SMPSO"]
#algorithms = ["MOEAD", "SPEA2", "NSGAII", "PESA2", "PAES", "EpsMOEA"]

### Pareto Dominance Functionality
- `weakly_dominated`: determines if a certain solution is dominated or non-dominated.
- `get_non_dominated_ix`: creates a column describing whether a certain solution is dominated or not.
- `add_dominated_col`: adds a `isDominated` column to the dataframe

In [6]:
def weakly_dominates(v0, v1):
    return np.all(v0 <= v1) and np.any(v0 < v1)

In [7]:
def get_non_dominated(V):
    nsols, nobjs = V.shape
    
    dominated = np.zeros((nsols, 1))
    dominated_by = np.zeros((nsols, 1))
    
    for i in range(nsols):
        for j in range(nsols):
            if i != j:
                if weakly_dominates(V[j], V[i]):
                    dominated[i] = 1
                    dominated_by[i] = j 
                    break
                    
    return dominated, dominated_by
        
def add_isdominated_cols(d, cols=objs_cols):
    A = np.array(d[cols])
    B, C = get_non_dominated(A)
    d["isDominated"] = pd.DataFrame(B, columns=["isDominated"])
    d["dominatedBy"] = pd.DataFrame(C, columns=["dominatedBy"])
    print(d["isDominated"].value_counts())

### Collect the Pareto Front for each algorithm (APFs)

In [8]:
# Collect data for each algorithm
dfs = [get_algorithm_data(alg) for alg in algorithms]

Reading file: final/outputs/algorithms/all_SMPSO_solutions.csv


In [9]:
# Add isDominated column for each algorithm
for i in range(len(algorithms)):
    print("\t\tAlgorithm", algorithms[i], "\nRetrieved", len(dfs[i]), "solutions")
    dfs[i].drop(drop_cols, axis=1, inplace=True)
    add_isdominated_cols(dfs[i])
    dfs[i].sort_values(by=o1, inplace=True)

		Algorithm SMPSO 
Retrieved 675 solutions
1.0    642
0.0     33
Name: isDominated, dtype: int64


In [10]:
# Get Non Dominated Solutions
APFs = [df[df['isDominated'] == 0] for df in dfs]

In [ ]:
i = algorithms.index("GDE3")
APFs[i]

# moead: 1.192813	5.516787	-0.883130	20.340443	-1.479601	23.575989	1.038524 objs: 1.038524    55.978605
# smpso: 1.570796	31.415927	-1.570796	31.415927	1.454480	5.826618	0.576629	78.072770

### Collect True Pareto Front

In [11]:
def get_true_pf_data(folder="final/outputs/"):
    file = folder + "all_solutions.csv"
    return pd.read_csv(file, sep=',', header=None)

def process_pf(d):
    d.drop(drop_cols, axis=1, inplace=True)
    add_isdominated_cols(d)

In [19]:
TPF = get_true_pf_data()
process_pf(TPF)
print("True Pareto Front")
TPF.describe()
TPF.sort_values(by=o1, inplace=True)

1.0    5556
0.0      33
Name: isDominated, dtype: int64
True Pareto Front


In [20]:
TPF.sort_values(by=o1, inplace=True)
TPF.head()

,3,4,5,6,7,8,9,10,isDominated,dominatedBy
2144,1.570796,31.415927,-1.570796,31.415927,1.454480,5.826618,0.576629,78.072770,0.0,0.0
4102,1.514007,31.415927,-1.502658,30.734539,-1.570796,31.415927,0.600049,40.928094,0.0,0.0
2007,1.570796,31.415927,1.570796,7.729243,-1.570796,31.415927,0.600684,74.687634,1.0,4102.0
2074,1.570796,31.415927,1.529704,10.383558,-1.570796,31.300227,0.603422,69.973513,1.0,4102.0
2288,1.570796,31.415927,-1.570796,31.415927,1.570796,8.408968,0.607136,73.491710,1.0,2074.0


# Visual Processing

In [21]:
import plotly
import plotly.plotly as py
import plotly.graph_objs as go
plotly.__version__

'3.7.1'

In [22]:
# plotly.tools.set_credentials_file(username='PastelBelem8', api_key='X8k1mloXRlB24rZm0qyq')
plotly.tools.set_credentials_file(username='KatarinaBelem', api_key='u5cJ4twW7lnZi128Bgjm')

In [15]:
colors = [
    # Verde
    'rgb(81, 142, 12)',
    # Vermelho
    'rgb(165, 0, 0)',
    # Roxo
    'rgb(152, 64, 188)',
    # Azul 
    'rgb(13, 188, 188)', 
    # Amarelo
    'rgb(255, 165, 21)',
    # Rosa
    'rgb(219, 31, 152)',
    # Cinzento claro
    'rgb(119, 130, 118)',
    # Azul escuro
    'rgb(0, 0, 255)',
    # Castanho
    'rgb(128, 64, 0)',
    # Preto
    'rgb(77, 38, 0)'
    ]

In [16]:
layout = go.Layout(
    # title = "Solutions of the minimization of two goals of an arc-shaped space frame",
    showlegend=True,
    # showlegend=False,

    xaxis=dict(
        title="Maximum Displacement",
        range=[0.57, 1.42],
        # autorange=True,
        showgrid=True,
        zeroline=False,
        showline=True,
        ticks='',
        showticklabels=True
    ),
    yaxis=dict(
        title="Euclidean Distance",
        range=[0, 80],
        # autorange=True,        
        showgrid=True,
        zeroline=False,
        showline=True,
        ticks='',
        showticklabels=True
    )
    )

In [23]:
def create_pf(algs=algorithms, colors=colors, layout=layout, approximatedPFs=APFs, combinedPf=TPF):
    # Create traces for each algorithm
    traces = []
    for i, a in enumerate(algorithms):
        trace = go.Scatter(
                    x = approximatedPFs[i][o1],
                    y = approximatedPFs[i][o2],
                    # mode = 'markers',
                    mode = 'lines+markers',
                    name = a + " Pareto Front",
                    marker=dict(
                        size=5,
                        color = colors[i],
                        opacity=1
                    ),
                    line=dict(
                        color = colors[i]
                    ))
        traces += [trace]
     
    if combinedPf is not None:
        tpf_o1 = combinedPf[combinedPf['isDominated'] == 0][o1]
        tpf_o2 = combinedPf[combinedPf['isDominated'] == 0][o2]
    
        # Add True PF
        tpf_trace = go.Scatter(
            x = tpf_o1,
            y = tpf_o2,
            mode = 'lines+markers',
            name = "Combined Pareto Front",
            marker=dict(
                size=6,
                color = 'rgb(13, 188, 188)', #'rgb(16, 46, 51)',
                opacity=1
                #line = dict(
                #     color = 'rgb(255, 255, 255)',
                #      width = 0.25
                # )
                )
            )
        traces.append(tpf_trace)
    fig = go.Figure(data=traces, layout=layout)
    return py.iplot(fig, filename='AllAlgorithmsPfs')

In [24]:
create_pf()

# All the points...

In [1]:
def create_dispersion_plot(algorithms=algorithms, colors=colors, layout=layout, data=dfs, combinedPf=TPF):
    # Create traces for each algorithm
    traces = []
    for i, a in enumerate(algorithms):
        datum = data[i]
        trace1 = go.Scatter(
                    x = datum[datum['isDominated'] == 0][o1],
                    y = datum[datum['isDominated'] == 0][o2],
                    #legendgroup = a,
                    name = a + " Non-dominated",
                    mode = 'lines+markers',
                    marker=dict(
                        size=5,
                        color = colors[i],
                        opacity=1
                    ),
                    line=dict(
                        color = colors[i]
                    ))
        trace2 = go.Scatter(
                    x = datum[datum['isDominated'] == 1][o1],
                    y = datum[datum['isDominated'] == 1][o2],
                    # legendgroup = a,
                    name = a + " dominated",
                    mode = 'markers',
                    marker=dict(
                        size=2.5,
                        color = colors[i],
                        opacity=0.8
                    ))

        traces += [trace1, trace2]
    if combinedPf is not None:
        tpf_o1 = combinedPf[combinedPf['isDominated'] == 0][o1]
        tpf_o2 = combinedPf[combinedPf['isDominated'] == 0][o2]

        tpf_trace = go.Scatter(
            x = tpf_o1,
            y = tpf_o2,
            mode = 'markers',
            name = "Combined Pareto Front",
            marker=dict(
                size=6,
                color = 'rgb(16, 46, 51)',
                opacity=1
                #line = dict(
               #     color = 'rgb(255, 255, 255)',
              #      width = 0.25
                 # )
                )
        )
        traces.append(tpf_trace)
    fig = go.Figure(data=traces, layout=layout)
    return py.iplot(fig, filename='ALL-TOGETHER-NOW')

NameError: name 'algorithms' is not defined

In [ ]:
create_dispersion_plot()

Contrastingly, when considering HV, if we compute the average of each run, PAES exhibits the largest volume,
hence strongly indicating that it is the best algorithm, with SMPSO, OMOPSO and MOEA/D
having the worst behaviors. However, when considering the combination of the three runs,
this situation is reversed and SMPSO and OMOPSO exhibit the best behaviors.

In [ ]:
df_extra_spea2 = df_extra_spea2.convert_objects(convert_numeric=True)
df_extra_spea2.info()

In [ ]:
# Additional run of SPEA2 for 1250 evaluations (50 generations of 25 individuals)
# df_extra_spea2_original = pd.read_csv("final/SPEA2_2000.csv", sep=',')
# Additional run of SPEA2 for 4000 evaluations (40 generations of 100 individuals)
df_extra_spea2_original = pd.read_csv("final/SPEA2_4000.csv", sep=',')

df_extra_spea2_original.drop(["Total time (s)", "Time o1", "Time o2"], axis=1, inplace=True)
df_extra_spea2 = df_extra_spea2_original.copy()

In [ ]:
df_extra_spea2[(df_extra_spea2["O1"] < 1) & (df_extra_spea2["O2"] < 60)].sort_values(by="O2")

In [ ]:
#df_extra_spea2.sort_values(by="O1", inplace=True)
add_isdominated_cols(df_extra_spea2, cols=["O1", "O2"])

In [ ]:
df_extra_spea2.sort_values(by="O1", inplace=True)
df_extra_spea2.head()

In [ ]:
traces3 = []
df_extra_spea2.sort_values(by="O1", inplace=True)
trace_extra_3 = go.Scatter(
                x = df_extra_spea2[df_extra_spea2['isDominated'] == 0]["O1"],
                y = df_extra_spea2[df_extra_spea2['isDominated'] == 0]["O2"],
                legendgroup = "Extra Spea2",
                name = "Extra SPEA2-4000 Non-dominated",
                mode = 'lines+markers',
                marker=dict(
                    size=5,
                    color = "rgb(0, 255, 0)",
                    opacity=1
                ),
                line=dict(
                    color = "rgb(0, 255, 0)"
                ))
trace_extra_4 = go.Scatter(
                x = df_extra_spea2[df_extra_spea2['isDominated'] == 1]["O1"],
                y = df_extra_spea2[df_extra_spea2['isDominated'] == 1]["O2"],
                legendgroup = "Extra Spea2",
                name = "Extra SPEA2-4000 dominated",
                mode = 'markers',
                marker=dict(
                    size=2.5,
                    color = "rgb(0, 255, 0)",
                    opacity=0.8
                ))

In [ ]:
traces3 = traces2 + [trace_extra_3, trace_extra_4]

In [ ]:
fig = go.Figure(data=traces3)
py.iplot(fig, filename='Extra Run')

In [ ]:
df_extra_spea2[(df_extra_spea2["O1"] <= 0.95) & (df_extra_spea2["O2"] <= 21)]

In [ ]:
df_extra_spea2.iloc[93:94]

In [ ]:
# Additional run of SPEA2 for 1250 evaluations (50 generations of 25 individuals)
# df_extra_spea2_original = pd.read_csv("final/SPEA2_2000.csv", sep=',')
# Additional run of SPEA2 for 4000 evaluations (40 generations of 100 individuals)
df_extra_smpso_original = pd.read_csv("final/outputs/all_SMPSO_2_solutions.csv", sep=',', header=None)
df_extra_smpso_original.head()

In [ ]:
#df_extra_spea2.sort_values(by="O1", inplace=True)
add_isdominated_cols(df_extra_smpso_original)

In [ ]:
traces4 = []
df_extra_smpso_original.sort_values(by=9, inplace=True)
trace_extra_5 = go.Scatter(
                x = df_extra_smpso_original[df_extra_smpso_original['isDominated'] == 0][o1],
                y = df_extra_smpso_original[df_extra_smpso_original['isDominated'] == 0][o2],
                legendgroup = "Extra SMPSO",
                name = "Extra SMPSO Non-dominated",
                mode = 'lines+markers',
                marker=dict(
                    size=5,
                    color = "rgb(255, 0, 0)",
                    opacity=1
                ),
                line=dict(
                    color = "rgb(255, 0, 0)"
                ))
trace_extra_6 = go.Scatter(
                x = df_extra_smpso_original[df_extra_smpso_original['isDominated'] == 1][o1],
                y = df_extra_smpso_original[df_extra_smpso_original['isDominated'] == 1][o2],
                legendgroup = "Extra SMPSO",
                name = "Extra SMPSO dominated",
                mode = 'markers',
                marker=dict(
                    size=2.5,
                    color = "rgb(255, 0, 0)",
                    opacity=0.8
                ))

In [ ]:
traces4 = traces3 + [trace_extra_5, trace_extra_6]

In [ ]:
fig = go.Figure(data=traces4)
py.iplot(fig, filename='Extra Run')

###  Check Min and Max Values per iteration...

In [ ]:
pop_size = 25
nrows=df_extra_spea2.shape[0]
nrows

In [ ]:
n_iterations = nrows // pop_size
n_iterations

In [ ]:
df_extra_spea2_original.iloc[0:2] # Syntax Example

In [ ]:
df_iterations = []
for i in range(n_iterations + 1):
    i_in, i_end = i * pop_size, i*pop_size+1
    df_iterations += [df_extra_spea2_original.loc[i_in:i_end].copy()]

In [ ]:
len(df_iterations)

In [ ]:
df_iterations

In [ ]:
iterations_min = pd.DataFrame(columns=df_iterations[1].columns)
for iteration in df_iterations:
    iterations_min.append(iteration.min(), ignore_index=True)
iterations_min

In [ ]:
df_iterations

# Indicators Processing

In [ ]:
indicators_df = pd.read_csv("final/outputs/indicators.csv", sep=',')
indicators_df[(indicators_df["Algorithm"] == "PAES")|(indicators_df["Algorithm"] == "SMPSO")]

In [ ]:
indicators_df.describe()

In [ ]:
# Create new DF with mean and std deviation
indicators_mean = indicators_df.groupby(["Algorithm"]).mean()
indicators_mean.sort_values(by="IGD", inplace=False)[["IGD", "HV"]]

In [ ]:
indicators_std = indicators_df.groupby(["Algorithm"]).std()
indicators_std

In [ ]:
# HV --> The larger, the better
# ONVG --> The larger, the better (number of non-dominated solutions per run)

## Combined PFs indicators

In [ ]:
combined_indicators_df = pd.read_csv("final/outputs/indicators_3_runs_combined.csv", sep=',')
combined_indicators_df.sort_values(by="IGD", inplace=False)[["IGD", "HV", "Algorithm"]]

In [45]:
combined_indicators_df

NameError: name 'combined_indicators_df' is not defined

# Test Click Events
I believe this currently works on online settings, but I have tried in the offline scenario with no success.

In [56]:
tpf_o1 = TPF[TPF['isDominated'] == 0][o1]
tpf_o2 = TPF[TPF['isDominated'] == 0][o2]
    
tpf_vars = TPF[TPF['isDominated'] == 0][vars_cols]


In [57]:
# Add True PF
tpf_trace = go.Scatter(
    x = tpf_o1,
    y = tpf_o2,
    mode = 'lines+markers',
    customdata = tpf_vars,
    name = "Combined Pareto Front",
    marker=dict(
        size=6,
        color = 'rgb(13, 188, 188)', #'rgb(16, 46, 51)',
        opacity=1
        #line = dict(
        #     color = 'rgb(255, 255, 255)',
        #      width = 0.25
        # )
        )
)

In [58]:
f = go.FigureWidget([tpf_trace])

In [59]:
scatter = f.data[0]

In [60]:
f.layout.hovermode = 'closest'

In [73]:
# create our callback function
def update_point(trace, points, selector):
    print(points.point_inds)
    
    clickedPoints = [print(trace['customdata'][points.point_inds][:])
    
    #for i in points.point_inds:

scatter.on_click(update_point)

In [74]:
f

FigureWidget({
    'data': [{'customdata': array([[  1.57079633,  31.41592654,  -1.57079633,  31.41592654,   1…

[0]
[[  1.57079633  31.41592654  -1.57079633  31.41592654   1.4544804
    5.82661777]]
InputDeviceState(
    ctrl=False, 
    alt=False, 
    shift=False, 
    meta=False, 
    button=0, 
    buttons=1)
[0]
[[  1.57079633  31.41592654  -1.57079633  31.41592654   1.4544804
    5.82661777]]
InputDeviceState(
    ctrl=True, 
    alt=False, 
    shift=False, 
    meta=False, 
    button=0, 
    buttons=1)
[2]
[[  1.52347532  31.41592654  -1.57079633  31.41592654  -1.57079633
   31.41592654]]
InputDeviceState(
    ctrl=False, 
    alt=False, 
    shift=False, 
    meta=False, 
    button=0, 
    buttons=1)
[18]
[[ -1.09009581  21.12500714  -0.93744986  19.53343796  -0.85651068
   17.56171882]]
InputDeviceState(
    ctrl=False, 
    alt=False, 
    shift=False, 
    meta=False, 
    button=0, 
    buttons=1)
[2]
[[  1.52347532  31.41592654  -1.57079633  31.41592654  -1.57079633
   31.41592654]]
InputDeviceState(
    ctrl=False, 
    alt=False, 
    shift=False, 
    meta=False, 
    button=0